In [ ]:
!pip install skyfield

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from skyfield.api import load, EarthSatellite

In [ ]:
# load TLE data
def load_tle(tle_file_path):
    satellites = {}
    with open(tle_file_path, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            name = lines[i].strip()
            line1 = lines[i + 1].strip()
            line2 = lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name)
            satellites[name] = satellite
    return satellites

tle_file_path = 'active.txt'
satellites = load_tle(tle_file_path)

In [ ]:
# # feature engineering
# def calculate_features(satellite1, satellite2, time):
#   position1 = satellite1.at(time).position.km
#   position2 = satellite2.at(time).position.km
#   velocity1 = satellite1.at(time).velocity.km_per_s
#   velocity2 = satellite2.at(time).velocity.km_per_s

#   distance = np.linalg.norm(position1 - position2)  # positional uncertainty
#   relative_velocity = np.linalg.norm(velocity1 - velocity2)  # relative velocity
#   inclination1 = satellite1.model.inclo  # orbital inclination (degrees)
#   inclination2 = satellite2.model.inclo
#   eccentricity1 = satellite1.model.ecco  # orbital eccentricity
#   eccentricity2 = satellite2.model.ecco

#   return [distance, relative_velocity, inclination1, inclination2, eccentricity1, eccentricity2]

In [ ]:
# generate sequential trajectory data
def generate_trajectory_data(satellite1, satellite2, timesteps=10):
    ts = load.timescale()
    start_time = ts.now()

    trajectories = []
    for t in range(timesteps):
      time = start_time + t*60 # increment time by 1 min

      try:
        position1 = satellite1.at(time).position.km
        position2 = satellite2.at(time).position.km
        velocity1 = satellite1.at(time).velocity.km_per_s
        velocity2 = satellite2.at(time).velocity.km_per_s

        # features
        distance = np.linalg.norm(position1 - position2)  # positional uncertainty
        relative_velocity = np.linalg.norm(velocity1 - velocity2)  # relative velocity

        inclination1 = satellite1.model.inclo if hasattr(satellite1.model, 'inclo') else 0
        inclination2 = satellite2.model.inclo if hasattr(satellite2.model, 'inclo') else 0
        eccentricity1 = satellite1.model.ecco if hasattr(satellite1.model, 'ecco') else 0
        eccentricity2 = satellite2.model.ecco if hasattr(satellite2.model, 'ecco') else 0

        trajectories.append([
            distance, relative_velocity, inclination1, inclination2, eccentricity1, eccentricity2
        ])

      except Exception as e:
        print(f"Error generating trajectory at timestep {t}: {e}")
        return None

    # ensure shape is always (timesteps, features)
    if len(trajectories) < timesteps:
      missing_timesteps = timesteps - len(trajectories)
      trajectories.extend([[0] * 6] * missing_timesteps)

    return np.array(trajectories)

In [ ]:
# simulated dataset
def generate_dataset(satellites, num_samples=500, timesteps=10):
    satellite_names = list(satellites.keys())
    data=[]
    labels=[]

    for _ in range(num_samples):
        sat1 = satellites[np.random.choice(satellite_names)]
        sat2 = satellites[np.random.choice(satellite_names)]
        trajectory = generate_trajectory_data(sat1, sat2, timesteps)
        if trajectory is None: # skip bad data
          continue

        collision = np.random.choice([0, 1], p=[0.95, 0.05]) # simulated label

        data.append(trajectory)
        labels.append(collision)

    return np.array(data), np.array(labels)

# generate trajectory data
timesteps=10
X, y = generate_dataset(satellites, num_samples=500, timesteps=timesteps)
print(np.isnan(X).any())

True


In [ ]:
# preprocess data
n_features = X.shape[2]
scalers = [StandardScaler() for _ in range(timesteps)]
for t in range(timesteps):
  X[:, t, :] = scalers[t].fit_transform(X[:, t, :])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# define LSTM  with hyperparameter tuning
def build_lstm_model(timesteps, n_features):
    model = Sequential([
        LSTM(64, input_shape=(timesteps, n_features), return_sequences=True, activation='tanh'),
        Dropout(0.3),
        LSTM(32, activation='tanh'),
        Dropout(0.3),
        Dense(1, activation='sigmoid') # binary classification
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=1.0), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_lstm_model(timesteps, n_features)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.8405 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9717 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9669 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9765 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9480 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9637 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9667 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9635 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('units_1', min_value=32, max_value=128, step=16),
        input_shape=(timesteps, n_features),
        return_sequences=True,
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(LSTM(
        units=hp.Int('units_2', min_value=32, max_value=128, step=16),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))


    model.add(Dense(1, activation='sigmoid'))


    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Hyperparameter tuning
import keras_tuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=2,
    directory='lstm_hyperparameter_tuning',
    project_name='satellite_collision_prediction'
)

# Run hyperparameter search
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first LSTM layer is {best_hps.get('units_1')},
the number of units in the second LSTM layer is {best_hps.get('units_2')},
the optimal dropout rates are {best_hps.get('dropout_1')} and {best_hps.get('dropout_2')},
and the best learning rate is {best_hps.get('learning_rate')}.
""")

Trial 20 Complete [00h 00m 24s]
val_accuracy: 0.9700000286102295

Best val_accuracy So Far: 0.9700000286102295
Total elapsed time: 00h 07m 24s

The optimal number of units in the first LSTM layer is 96,
the number of units in the second LSTM layer is 96,
the optimal dropout rates are 0.30000000000000004 and 0.4,
and the best learning rate is 0.001.



In [ ]:
# Use best hyperparameters found
final_model = tuner.hypermodel.build(best_hps)
final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate the model
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"\nFinal Model Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.8884 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9429 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9734 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9719 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9608 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9633 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9701 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9607 - loss: nan - val_accuracy: 0.9700 - val_loss: nan
Epoch 9/30
13/13 ━━━━━━━━━━━━━━━

In [ ]:
# Real-time collision prediction to predict risk for 2 satellites
sat1 = satellites[list(satellites.keys())[0]]
sat2 = satellites[list(satellites.keys())[1]]

trajectory = generate_trajectory_data(sat1, sat2, timesteps)
for t in range(timesteps):
    trajectory[t, :] = scalers[t].transform(trajectory[t, :].reshape(1, -1))

trajectory = np.expand_dims(trajectory, axis=0)  # Add batch dimension
collision_prob = model.predict(trajectory)[0][0]
print(f"\nPredicted Collision Probability: {collision_prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step

Predicted Collision Probability: nan
